## Preambule

In [7]:
import logging
import warnings

import numpy as np
from tqdm import tqdm

warnings.simplefilter(action="ignore", category=FutureWarning)

logging.basicConfig(level="INFO", format="%(message)s")
# See https://github.com/Textualize/rich/pull/3329 for nicer logs in notebooks

%load_ext autoreload
%autoreload all

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Run classes

Data reader class

In [9]:
import effortsharing as es

# Load configuration
config = es.Config.from_file('config.yml')

# Read input data
countries, regions = es.input.socioeconomics.read_general(config)
socioeconomic_data = es.input.socioeconomics.load_socioeconomics(config)
modelscenarios = es.input.emissions.read_modelscenarios(config)
emission_data = es.input.emissions.load_emissions(config)
primap_data = es.input.emissions.read_primap(config)
ndc_data = es.input.ndcs.load_ndcs(config, emission_data)

# Calculate global budgets and pathways
xr_temperatures, xr_nonco2warming_wrt_start = es.pathways.nonco2.nonco2variation(config)
xr_traj_nonco2 = es.pathways.nonco2.determine_global_nonco2_trajectories(config)
xr_co2_budgets = es.pathways.global_budgets.determine_global_budgets(config)
all_projected_gases = es.pathways.co2_trajectories.determine_global_co2_trajectories(config)

# Merge all data into a single xrarray object
xr_total = (
    es.main.merge_data(
        xr_co2_budgets,
        all_projected_gases,
        emission_data,  # TODO: already stored elsewhere. Skip?
        ndc_data,  # TODO: already stored elsewhere. Skip?
        socioeconomic_data,  # TODO: already stored elsewhere. Skip?
    )
    .reindex(Region=list(regions.values()))
    .reindex(Time=np.arange(1850, 2101))
    .interpolate_na(dim="Time", method="linear")
)

# Add country groups
new_total, new_regions = es.main.add_country_groups(config, regions, xr_total)

# Save the data
save_temp = np.array(config.dimension_ranges.peak_temperature_saved).astype(float).round(2)
xr_version = new_total.sel(Temperature=save_temp)

es.main.save_regions(config, new_regions, countries)
es.main.save_total(config, xr_version)
es.main.save_rbw(config, xr_version, countries)
es.main.save_rci(config, xr_version)

# Country-specific data readers
es.country_specific.netherlands.datareader_netherlands(config, new_total)
es.country_specific.norway.datareader_norway(config, new_total, primap_data)


Reading unfccc country data
Loading intermediate data from K:\Code\effort-sharing\data\intermediate\socioeconomics.nc
Loading intermediate data from K:\Code\effort-sharing\data\intermediate\ar6_modelscenarios.json
Loading intermediate data from K:\Code\effort-sharing\data\intermediate\emissions_all.nc
Loading intermediate data from K:\Code\effort-sharing\data\intermediate\primap.nc
Loading intermediate data from K:\Code\effort-sharing\data\intermediate\ndcs.nc
Computing global nonco2 trajectories
Loading intermediate data from K:\Code\effort-sharing\data\intermediate\emissions_all.nc
Loading intermediate data from K:\Code\effort-sharing\data\intermediate\ar6_modelscenarios.json
Loading intermediate data from K:\Code\effort-sharing\data\intermediate\global_co2_budgets.nc
Computing global co2 trajectories
Loading intermediate data from K:\Code\effort-sharing\data\intermediate\emissions_all.nc
Loading intermediate data from K:\Code\effort-sharing\data\intermediate\ar6_modelscenarios.json


In [6]:
from effortsharing.allocation import allocation

allocator = allocation("USA", lulucf="incl", gas="GHG")
allocator.gf()
allocator.pc()
allocator.pcc()
allocator.pcb()
allocator.ecpc()
allocator.ap()
allocator.gdr()
allocator.save()

FileNotFoundError: [Errno 2] No such file or directory: 'input.yml'

Allocations specifically for Norway and Netherlands if you want to harmonize to national data

In [ ]:
from effortsharing.allocation import allocation

for lulucf in ["incl", "excl"]:
    for gas in ["GHG", "CO2"]:
        for dataread_file in [
            "xr_dataread.nc",
            "xr_dataread_nor.nc",
        ]:  # This is only necessary if for a country a specific historical emissions profile is required beyond what is in Jones (e.g. Norway was requested)
            allocator = allocation("NOR", lulucf=lulucf, gas=gas, dataread_file=dataread_file)
            allocator.gf()
            allocator.pc()
            allocator.pcc()
            allocator.pcb()
            allocator.ecpc()
            allocator.ap()
            allocator.gdr()
            allocator.save()
            print("NOR", lulucf, gas, dataread_file, "done")

NOR incl GHG xr_dataread.nc done
NOR incl GHG xr_dataread_nor.nc done
NOR incl CO2 xr_dataread.nc done
NOR incl CO2 xr_dataread_nor.nc done
NOR excl GHG xr_dataread.nc done
NOR excl GHG xr_dataread_nor.nc done
NOR excl CO2 xr_dataread.nc done
NOR excl CO2 xr_dataread_nor.nc done


Allocation class (run all 207 countries)

In [ ]:
from effortsharing.allocation import allocation

regions_iso = np.load(
    "K:/Data/Data_effortsharing/DataUpdate_ongoing/all_regions.npy", allow_pickle=True
)
for cty in tqdm(regions_iso):
    allocator = allocation(cty, lulucf="incl", gas="GHG")
    allocator.gf()
    allocator.pc()
    allocator.pcc()
    allocator.pcb()
    allocator.ecpc()
    allocator.ap()
    allocator.gdr()
    allocator.save()

100%|██████████| 207/207 [2:42:29<00:00, 47.10s/it]  


Allocation rules combining approaches (Robiou paper)

In [ ]:
from effortsharing.allocation_combinedapproaches import allocation_comb

allocator = allocation_comb()
allocator.ecpc()
allocator.pc()
allocator.discounting_historical_emissions()
allocator.approach1gdp()
allocator.approach1hdi()
allocator.approach2()
allocator.approach2_transition()
allocator.combine()
# allocator.get_relation_2030emis_temp()
allocator.determine_tempoutcomes_discrete()
allocator.save()

Temperature NDC-alignment metric

In [ ]:
from effortsharing.tempalign import tempaligning

tempaligner = tempaligning()  # FIRST RUN AGGREGATOR FOR THIS!! (2030 alloc)
tempaligner.get_relation_2030emis_temp()
tempaligner.determine_tempoutcomes()
tempaligner.save()

# ==================================== #
# Initializing tempaligning class        #
# ==================================== #
- Determine relation between 2030-emissions and temperature outcome
- Determine temperature metric


100%|██████████| 6/6 [00:11<00:00,  1.96s/it]


- Save


Reading policy scenarios from ENGAGE

In [ ]:
from effortsharing.policyscens import policyscenadding

policyscenner = policyscenadding()
policyscenner.read_engage_data()
policyscenner.filter_and_convert()
policyscenner.add_to_xr()

# ==================================== #
# Initializing policyscenadding class  #
# ==================================== #
- Read ENGAGE scenarios and change region namings
- Filter correct scenarios and convert to xarray object
- Add to overall xrobject


Variance decomposition

In [ ]:
from effortsharing.variancedecomp import vardecomposing

vardecomposer = vardecomposing(startyear=2021, gas="GHG", lulucf="incl")
vardecomposer.sobolindices = {}
print("- Starting sobols for fixed years, over many countries")
timeseries = np.arange(2025, 2101, 1)
for year in timeseries:
    print("  Starting with", year)
    xr_cty, ar_time, array_dims, array_inputs, problem, samples = (
        vardecomposer.prepare_global_sobol(year)
    )
    vardecomposer.sobolindices[year] = vardecomposer.apply_decomposition(
        xr_cty, ar_time, array_dims, array_inputs, problem, samples
    )
vardecomposer.save(array_dims, timeseries)

# ==================================== #
# Initializing vardecomposing class    #
# ==================================== #
- Starting sobols for fixed years, over many countries
  Starting with 2025


207it [02:33,  1.35it/s]


  Starting with 2026


207it [02:33,  1.35it/s]


  Starting with 2027


...

  Starting with 2099


207it [02:25,  1.42it/s]


  Starting with 2100


207it [02:24,  1.43it/s]

- Save global results


In [ ]:
vardecomposer.save(array_dims, timeseries)

- Save global results
